In [1]:
#@title ## Setup
# !py -m pip install ultralyticsplus
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.239 🚀 Python-3.10.0 torch-2.2.2+cu121 CPU (Intel Core(TM) i5-4460  3.20GHz)
Setup complete ✅ (4 CPUs, 18.7 GB RAM, 48.3/251.0 GB disk)


In [14]:
# !py -m pip install roboflow
import json
import os
import sys
import time
import glob
from getpass import getpass
from urllib.parse import urlparse

import numpy as np
from numpy import ndarray
from roboflow import Roboflow, load_roboflow_api_key
from roboflow.core.workspace import Workspace, count_comparisons, check_box_size
from roboflow.config import DEMO_KEYS
from roboflow.adapters import rfapi

class WorkspaceLocalhost(Workspace):
    def active_learning(
        self,
        raw_data_location: str = "",
        raw_data_extension: str = "",
        inference_endpoint: list = [],
        upload_destination: str = "",
        conditionals: dict = {},
        use_localhost: bool = False,
    ) -> str:
        """perform inference on each image in directory and upload based on conditions
        @params:
            raw_data_location: (str) = folder of frames to be processed
            raw_data_extension: (str) = extension of frames to be processed
            inference_endpoint: (List[str, int]) = name of the project
            upload_destination: (str) = name of the upload project
            conditionals: (dict) = dictionary of upload conditions
            use_localhost: (bool) = determines if local http format used or remote endpoint
        """  # noqa: E501 // docs
        prediction_results = []

        # ensure that all fields of conditionals have a key:value pair
        conditionals["target_classes"] = [] if "target_classes" not in conditionals else conditionals["target_classes"]
        conditionals["confidence_interval"] = (
            [30, 99] if "confidence_interval" not in conditionals else conditionals["confidence_interval"]
        )
        conditionals["required_class_variance_count"] = (
            1 if "required_class_variance_count" not in conditionals else conditionals["required_class_variance_count"]
        )
        conditionals["required_objects_count"] = (
            1 if "required_objects_count" not in conditionals else conditionals["required_objects_count"]
        )
        conditionals["required_class_count"] = (
            0 if "required_class_count" not in conditionals else conditionals["required_class_count"]
        )
        conditionals["minimum_size_requirement"] = (
            float("-inf")
            if "minimum_size_requirement" not in conditionals
            else conditionals["minimum_size_requirement"]
        )
        conditionals["maximum_size_requirement"] = (
            float("inf") if "maximum_size_requirement" not in conditionals else conditionals["maximum_size_requirement"]
        )

        # check if inference_model references endpoint or local
        local = "http://172.20.208.1:9001/" if use_localhost else None

#         inference_model = (
#             self.project(inference_endpoint[0]).version(version_number=inference_endpoint[1], local=local).model
#         )

#         rf = Roboflow(api_key="tGo2415ulPekOjTMO3Ts")
#         project = rf.workspace("303-tywrd").project(inference_endpoint[0])
#         inference_model = project.version(version_number=inference_endpoint[1]).model

        # infer on a local image
#         print(model.predict("your_image.jpg", confidence=40, overlap=30).json())

        # hugging-face
        from ultralyticsplus import YOLO, download_from_hub

        hub_model_id = "chanelcolgate/chamdiemgianhang-vsk-v5" # @param {type:"string"}

        # Load a model
        model_path = download_from_hub(hub_model_id)
        inference_model = YOLO(model_path)
    
    
        upload_project = self.project(upload_destination)

#         print("inference reference point: ", inference_model)
        print("upload destination: ", upload_project)

        # check if raw data type is cv2 frame
        if issubclass(type(raw_data_location), np.ndarray):
            globbed_files = [raw_data_location]
        else:
            globbed_files = glob.glob(raw_data_location + "/*" + raw_data_extension)

        image1 = globbed_files[0]
        similarity_timeout_counter = 0

        for index, image in enumerate(globbed_files):
            try:
                print(
                    "*** Processing image [" + str(index + 1) + "/" + str(len(globbed_files)) + "] - " + image + " ***"
                )
            except Exception:
                pass

            if "similarity_confidence_threshold" in conditionals.keys():
                image2 = image
                # measure the similarity of two images using CLIP (hits an endpoint hosted by Roboflow)   # noqa: E501 // docs
                similarity = clip_encode(image1, image2, CLIP_FEATURIZE_URL)
                similarity_timeout_counter += 1

                if (
                    similarity <= conditionals["similarity_confidence_threshold"]
                    or similarity_timeout_counter == conditionals["similarity_timeout_limit"]
                ):
                    image1 = image
                    similarity_timeout_counter = 0
                else:
                    print(image2 + " --> similarity too high to --> " + image1)
                    continue  # skip this image if too similar or counter hits limit

            # predictions = inference_model.predict(image).json()["predictions"]
            import json
            predictions = json.loads(inference_model.predict(image)[0].tojson())
#             print(predictions)
            for prediction in predictions:
                prediction.update({
                    "class_id": int(prediction["class"]),
                    "class": prediction["name"]}
                )
                
            # collect all predictions to return to user at end
            prediction_results.append({"image": image, "predictions": predictions})

            # compare object and class count of predictions if enabled,
            # continue if not enough occurrences
            if not count_comparisons(
                predictions,
                conditionals["required_objects_count"],
                conditionals["required_class_count"],
                conditionals["target_classes"],
            ):
                print(" [X] image failed count cases")
                continue

            # iterate through all predictions
            for i, prediction in enumerate(predictions):
                print(i)

                # check if box size of detection fits requirements
#                 if not check_box_size(
#                     prediction,
#                     conditionals["minimum_size_requirement"],
#                     conditionals["maximum_size_requirement"],
#                 ):
#                     print(" [X] prediction failed box size cases")
#                     continue

                # compare confidence of detected object to confidence thresholds
                # confidence comes in as a .XXX instead of XXX%
                if (
                    prediction["confidence"] * 100 >= conditionals["confidence_interval"][0]
                    and prediction["confidence"] * 100 <= conditionals["confidence_interval"][1]
                ):
                    # filter out non-target_class uploads if enabled
                    if (
                        len(conditionals["target_classes"]) > 0
                        and prediction["class"] not in conditionals["target_classes"]
                    ):
                        print(" [X] prediction failed target_classes")
                        continue

                    # upload on success!
                    print(" >> image uploaded!")
                    upload_project.upload(image, num_retry_uploads=3)
                    import shutil
                    shutil.move(image, image.replace('dataset', 'unlabeled28'))
                    break

        # return predictions with filenames if globbed images from dir,
        # otherwise return latest prediction result
        return prediction_results if type(raw_data_location) is not ndarray else prediction_results[-1]["predictions"]
    
class RoboflowLocalhost(Roboflow):
    def workspace(self, the_workspace=None):
        sys.stdout.write("\r" + "loading Roboflow workspace...")
        sys.stdout.write("\n")
        sys.stdout.flush()

        if the_workspace is None:
            the_workspace = self.current_workspace

        if self.api_key in DEMO_KEYS:
            return Workspace({}, self.api_key, the_workspace, self.model_format)
        workspace_api_key = load_roboflow_api_key(the_workspace)
        api_key = workspace_api_key or self.api_key

        list_projects = rfapi.get_workspace(api_key, the_workspace)
        return WorkspaceLocalhost(list_projects, api_key, the_workspace, self.model_format)


In [15]:
# dataset 1
import os
from roboflow import Roboflow

rf = RoboflowLocalhost(api_key="tGo2415ulPekOjTMO3Ts")
workspace = rf.workspace("303-tywrd")

raw_data_location = './dataset/'
raw_data_extension = ".jpg"

inference_endpoint = ["vsk-1", 1]
upload_destination = "vsk-1"

# conditionals = {
#     # "similarity_confidence_threshold": 0
#     "confidence_interval": [10, 90],
#     "target_classes": ["BOM_GEN", "BOM_JUN", "BOM_KID", "BOM_SAC", "BOM_VTG", "BOM_YTV",
#                        "TUI_GEN", "TUI_JUN", "TUI_KID", "TUI_SAC", "TUI_THV", "TUI_THX",
#                        "TUI_VTG", "TUI_YTV"],
# }

conditionals = {
    # "similarity_confidence_threshold": 0
    "confidence_interval": [10, 90],
    "target_classes": ["BOM_GEN", "TUI_GEN"],
}

workspace.active_learning(
    raw_data_location=raw_data_location,
    raw_data_extension=raw_data_extension,
    inference_endpoint=inference_endpoint,
    upload_destination=upload_destination,
    conditionals=conditionals,
    use_localhost=True
)

In [ ]:
# dataset 2
import os
from roboflow import Roboflow

rf = RoboflowLocalhost(api_key="tGo2415ulPekOjTMO3Ts")
workspace = rf.workspace("303-tywrd")

raw_data_location = './dataset/'
raw_data_extension = ".jpg"

inference_endpoint = ["vsk-1", 1]
upload_destination = "vsk-1"

conditionals = {
    # "similarity_confidence_threshold": 0
    "confidence_interval": [10, 90],
    "target_classes": ["BOM_GEN", "BOM_JUN", "BOM_KID", "BOM_SAC", "BOM_VTG", "BOM_YTV",
                       "TUI_GEN", "TUI_JUN", "TUI_KID", "TUI_SAC", "TUI_THV", "TUI_THX",
                       "TUI_VTG", "TUI_YTV"],
}

workspace.active_learning(
    raw_data_location=raw_data_location,
    raw_data_extension=raw_data_extension,
    inference_endpoint=inference_endpoint,
    upload_destination=upload_destination,
    conditionals=conditionals,
    use_localhost=True
)